# kmeans - transformacja danych

Do zmiany układu współrzędnych (redukcji wymiarowości) używane jest zwykle SVD. Wynik grupowania można wykorzystać do transformacji (nieliniowej) danych. Idea podobna jest do metod kernelowych.


ZADANIE: Proszę wziąć dane ecoli z UCI. Podzielić na zbiór testowy i treningowy.

1. Uczymy logistyczną regresję i testujemy (accuracy)

2. Na zbiorze treningowym robimy k-means (5 klastrów) co daje podział. Dla każdego punktu liczymy odległość od każdego klastra. Odległości dają nam nowe współrzędne (tyle ile klastrów). Na takiej reprezentacji uczymy logistyczną regresję.

Proszę rozważyć odległość euklidesową do liczenia odległości od klastra jak również: $d(x,y) = 1-\exp(a \|x-y\|^2)$, gdzie $a$ to parameter.

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    filepath_or_buffer='data_cl/ecoli.dat', 
    header=None, 
    sep=',')

X = np.array(df.iloc[:, :-1])
X
print(X.shape)
y = df.iloc[:,-1]

labels = set(y)
labels
#{cp,im,imS,imL,imU,om,omL,pp}

(336, 7)


{'cp', 'im', 'imL', 'imS', 'imU', 'om', 'omL', 'pp'}

In [10]:
z = {'cp':0, 'im':1, 'imL':2, 'imS':3, 'imU':4, 'om':5, 'omL':6, 'pp':7}
z

{'cp': 0, 'im': 1, 'imL': 2, 'imS': 3, 'imU': 4, 'om': 5, 'omL': 6, 'pp': 7}

In [11]:
Y = np.array([z[key] for key in y])
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

In [12]:
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)
print(X_train.shape[0])
print(X_test.shape[0])

268
68


In [13]:
n = 5

from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=n, random_state = 42)
# slice matrix so we only include the 0/1 indicator columns in the clustering
clust_train = cluster.fit_predict(X_train)

clust_test = cluster.predict(X_test)

In [16]:
from scipy.spatial.distance import sqeuclidean
import math
a = 0.0001

def dist(X, centers, a):
    data = np.array([[1-sqeuclidean(X[i], centers[k]) for k in range(n)] for i in range(X.shape[0])])
    return data


# def dist(X, centers, a):
#     data = np.array([[1-math.exp(-a*sqeuclidean(X[i], centers[k])) for k in range(n)] for i in range(X.shape[0])])
#     return data



In [17]:
data_train = dist(X_train, cluster.cluster_centers_, a)
data_test = dist(X_test, cluster.cluster_centers_, a)

In [18]:
from sklearn.metrics import accuracy_score

logreg = linear_model.LogisticRegression(C=1e5, random_state= 42) 

logreg.fit(data_train, y_train)
y_train_pred = logreg.predict(data_train)
acc_train = accuracy_score(y_train, y_train_pred)
print("Dokładność na zbiorze treningowym ", acc_train)
    
y_test_pred = logreg.predict(data_test)
acc_test = accuracy_score(y_test, y_test_pred)
print("Dokładność na zbiorze testowym ", acc_test)

Dokładność na zbiorze treningowym  0.7649253731343284
Dokładność na zbiorze testowym  0.8382352941176471


In [19]:
logreg = linear_model.LogisticRegression(C=1e5, random_state = 42) # w prakcy powinniśmy dopasować parametr c
logreg.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [20]:
from sklearn.metrics import accuracy_score

y_train_pred = logreg.predict(X_train)
print("Dokładność na zbiorze treningowym ", accuracy_score(y_train, y_train_pred))



y_test_pred = logreg.predict(X_test)
print("Dokładność na zbiorze test ", accuracy_score(y_test, y_test_pred))

Dokładność na zbiorze treningowym  0.7985074626865671
Dokładność na zbiorze test  0.8235294117647058
